# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG
- Task 6: Visibility Tooling
- Task 7: RAG Evaluation Using GPT-4

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/PvlaIUO.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [2]:
! pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv pypdf

In [3]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [4]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [5]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

2

In [6]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one document - and it's the entire text of Frakenstein

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

Let's take a peek visually at what we're doing here - and why it might be useful:

<img src="https://i.imgur.com/rtM6Ci6.png" />

As you can see (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [7]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

746

Let's take a look at some of the documents we've managed to split.

In [8]:
documents[0][:1006]

'\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and why

In [9]:
[i for i in split_documents if 'ty' ==i[:2]]

[]

In [10]:
split_documents[0:10]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format.

Loosely, this means turning the text into numbers.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### Embeddings and Dense Vector Search: A Quick Primer

If you come from an NLP background, embeddings are something you might be intimately familiar with - otherwise, you might find the topic a bit...dense. (this attempt at a joke will make more sense later)

In all seriousness, embeddings are a powerful piece of the NLP puzzle, so let's dive in!

> NOTE: While this notebook language/NLP-centric, embeddings have uses beyond just text!

#### Why Do We Even Need Embeddings?

In order to fully understand what Embeddings are, we first need to understand why we have them!

Machine Learning algorithms, ranging from the very big to the very small, all have one thing in common:

They need numeric inputs.

So we need a process by which to translate the domain we live in, dominated by images, audio, language, and more, into the domain of the machine: Numbers.

Another thing we want to be able to do is capture "semantic information" about words/phrases so that we can use algorithmic approaches to determine if words are closely related or not!

So, we need to come up with a process that does these two things well:

- Convert non-numeric data into numeric-data
- Capture potential semantic relationships between individual pieces of data

#### How Do Embeddings Capture Semantic Relationships?

In a simplified sense, embeddings map a word or phrase into n-dimensional space with a dense continuous vector, where each dimension in the vector represents some "latent feature" of the data.

This is best represented in a classic example:

![image](https://i.imgur.com/K5eQtmH.png)

As can be seen in the extremely simplified example: The X_1 axis represents age, and the X_2 axis represents hair.

The relationship of "puppy -> dog" reflects the same relationship as "baby -> adult", but dogs are (typically) hairier than humans. However, adults typically have more hair than babies - so they are shifted slightly closer to dogs on the X_2 axis!

Now, this is a simplified and contrived example - but it is *essentially* the mechanism by which embeddings capture semantic information.

In reality, the dimensions don't sincerely represent hard-concepts like "age" or "hair", but it's useful as a way to think about how the semantic relationships are captured.

Alright, with some history behind us - let's examine how these might help us choose relevant context.

Let's begin with a simple example - simply looking at how close to embedding vectors are for a given phrase.

When we use the term "close" in this notebook - we're referring to a distance measure called "cosine similarity".

We discussed above that if two embeddings are close - they are semantically similar, cosine similarity gives us a quick way to measure how similar two vectors are!

Closeness is measured from 1 to -1, with 1 being extremely close and -1 being extremely close to opposite in meaning.

Let's implement it with Numpy below.

In [11]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

Now let's use the `text-embedding-3-small` embedding model (more on that in a second) to embed two sentences. In order to use this embedding model endpoint - we'll need to provide our OpenAI API key!

In [12]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [13]:
from aimakerspace.openai_utils.embedding import EmbeddingModel

embedding_model = EmbeddingModel()

Let's define our two sentences:

In [14]:
puppy_sentence = "I love puppies!"
dog_sentence = "I love dogs!"

Now we can convert those into embedding vectors using OpenAI!

In [15]:
puppy_vector = embedding_model.get_embedding(puppy_sentence)
dog_vector = embedding_model.get_embedding(dog_sentence)

Now we can determine how closely they are related using our distance measure!

In [16]:
cosine_similarity(puppy_vector, dog_vector)

0.8340215662150423

Remember, with cosine similarity, close to 1. means they're very close!

Let's see what happens if we use a different set of sentences.

In [17]:
puppy_sentence = "I love puppies!"
cat_sentence = "I dislike cats!"

puppy_vector = embedding_model.get_embedding(puppy_sentence)
cat_vector = embedding_model.get_embedding(cat_sentence)

cosine_similarity(puppy_vector, cat_vector)

0.37243333229832976

As you can see - these vectors are further apart - as expected!

Now that we've gotten some background - lets see this put together with a few extra layers on top!

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?

    By modifying optional endpoint API parameter called "dimensions", but it's only supported in text-embedding-3 and later models. 
    
    Reference on [link](https://platform.openai.com/docs/api-reference/embeddings/create#embeddings-create-dimensions).

2. What technique does OpenAI use to achieve this?

    Matryoshka Representation Learning


> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [18]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

Async approach to collecting embeddings can improve performance due allowing other tasks to proceed while waiting for embedding to complete, scalability due ability to handle larger number of tasks on same system, resource efficiency..It can enhance overall robustness and responsiveness of a system.

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [19]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

Outputs are non-deterministic by default which means model will output different outputs from request to request. More reproducible outputs can be achieved by setting seed parameter in an API call and make them mostly deterministic. 

Reference on this [link](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs)

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-3.5-turbo`!

Let's tie all these together and use it to prompt `gpt-3.5-turbo`!

In [20]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]

response = chat_openai.run(messages)

In [21]:
print(response)

The best way to write a loop depends on the specific use case and the programming language you are using. In Python, one common way to write a loop is by using a for loop. Here is an example of a basic for loop in Python:

```python
for i in range(5):
    print(i)
```

This loop will iterate over the range of numbers from 0 to 4 and print each number.

Another common type of loop in Python is a while loop. Here is an example of a while loop in Python:

```python
i = 0
while i < 5:
    print(i)
    i += 1
```

This loop will run as long as the condition `i < 5` is true and will print the value of `i` in each iteration.

Remember to consider the requirements of your specific task when choosing the type of loop to use and make sure to structure your loop in a way that is clear and readable for others who may need to understand or modify your code in the future.


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [22]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_user_prompt, formatted_system_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

Chain of thought, few-shot examples, templates..

What is that strategy called?

Answered above, but we can use any of those or comination of those promting strategies to achieve more detailed responses.

> NOTE: You can look through the Week 1 Day 2 material for an answer to this question!

In [23]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [24]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")

{'response': 'The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or leader who was successful in a particular function (e.g., product management, sales, marketing) hires a weak executive to run that function in order to maintain control and continue being seen as the leader in that area. This can lead to ineffective leadership within that function and hinder overall organizational success.',
 'context': [('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV netw

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

In [25]:
# load pdf to rag

pdf_loader = TextFileLoader("data//fallout-man-eng.pdf")
documents = pdf_loader.load_documents()
len(documents)

text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)


vector_fallout_db = VectorDatabase()
vector__fallout_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [26]:
retrieval_augmented_qa_pipeline_fallout = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector__fallout_db,
    llm=chat_openai
)

In [27]:
retrieval_augmented_qa_pipeline_fallout.run_pipeline("What is starting budget of Vault-13?")

{'response': 'The starting budget of Vault-13 was $400,000,000,000.',
 'context': [('is with a description of your new home.\nIMPORTANT VAULT STATISTICS\nVault Number ............................13\nStarting construction date ....August 2063\nEnding construction date ......March 2069\nStarting Budget ........................$400,000,000,000\nFinal Budget, with interest ..$645,000,000,000\nTotal number of occupants ....1,000 (at capacity)\nTotal duration ..........................10 years (at capacity)\nNumber of living quarters ......100 (hot bunking required if at \nmaximum capacity)VDSG  –  RESTRICTED  –  VTB-OO1-13\nRESTRICTED 1—11\nDon’t let what could have happened bother you. We have\nenough problems to deal with in the here and now.\n- Overseer\n\nDoor thickness ..........................4 yards, steel\nEarth coverage ..........................3,200,000 tons of soil, at 200 feet\nComputer control system ......Think machine\nPrimary power supply ............Geo-thermal\nSecondary

## Task 6: Visibility Tooling

This is great, but what if we wanted to add some visibility to our pipeline?

Let's use Weights and Biases as a visibility tool!

The first thing we'll need to do is create a Weights and Biases account and get an API key.

You can follow the process outlined [here](https://docs.wandb.ai/quickstart) to do exactly that!

Now we can get the Weights and Biases dependency and add our key to our env. to begin!

In [28]:
! pip install -qU wandb

In [29]:
wandb_key = getpass("Weights and Biases API Key: ")
os.environ["WANDB_API_KEY"] = wandb_key

In [30]:
import wandb

wandb.init(project="Visibility Example - AIE3")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kbrdek (ai-bootcamp). Use `wandb login --relogin` to force relogin


Now we can integrate Weights and Biases into our `RetrievalAugmentedQAPipeline`.

```python
if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
```

The main things to consider here are how to populate the various fields to make sure we're tracking useful information.

We'll use the `text_only` flag to ensure we can get detailed information about our LLM call!

You can check out all the parameters for Weights and Biases `Trace` [here](https://github.com/wandb/wandb/blob/653015a014281f45770aaf43627f64d9c4f04a32/wandb/sdk/data_types/trace_tree.py#L166)

In [31]:
import datetime
from wandb.sdk.data_types.trace_tree import Trace

class RetrievalAugmentedGenerationPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, wandb_project = None) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.wandb_project = wandb_project

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)


        start_time = datetime.datetime.now().timestamp() * 1000

        try:
            openai_response = self.llm.run([formatted_system_prompt, formatted_user_prompt], text_only=False)
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "success"
            status_message = (None, )
            response_text = openai_response.choices[0].message.content
            token_usage = dict(openai_response.usage)
            model = openai_response.model

        except Exception as e:
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "error"
            status_message = str(e)
            response_text = ""
            token_usage = {}
            model = ""

        if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage,
                    "model_name" : model
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")

        return {"response" : self.llm.run([formatted_user_prompt, formatted_system_prompt]), "context" : context_list} if response_text else "We ran into an error. Please try again later. Full Error Message: " + status_message

In [32]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedGenerationPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    wandb_project="LLM Visibility Example"
)

In [33]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?")

{'response': "I don't know.",
 'context': [('pparel worn to defend against attacks and damage.  Betterarmor can protect against bigger attacks.\nThere are three factors that determine the quality of armor.\nArmor Class.  The first function of armor is to prevent the\nwearer from being hit by the attack.  This is known as Armor Class(or AC).  The higher the AC, the better the armor.  The AC subtractsfrom the to hit chance of the attacker.  Example: you are wearinglight leather armor, with an AC of 15%.  Your attacker has a base toVDSG  –  RESTRICTED  –  VTB-OO1-13\nRESTRICTED 5—12\n\nhit chance of 50% (assuming all the other modifiers have been taken\ninto account).  His final to hit chance is only (50% - 15%) 35%.\nDamage Threshold.   The second step is blocking the damag-\ning energy of the attack.  This is known as the Damage Threshold (orDT).  The DT is subtracted from the hit point damage of the attack.Example: Assume the above attacker succeeds.  His attack does 20points of damage

In [34]:
retrieval_augmented_qa_pipeline.run_pipeline("What are some tips for being an effective CEO?")

{'response': "Based on the provided context, here are some tips for being an effective CEO:\n\n1. Nuke ineffective managers and promote top performers: Identify and promote strong, sharp, ambitious managers to lead teams, giving them clear missions and incentives.\n\n2. Have a direct line to top talent: Keep only one layer of management between yourself and the top 20-30 up-and-coming leaders in your company.\n\n3. Focus on winning key objectives: Put your best person in charge of winning the most important aspect of your company's success.\n\n4. Use incentives strategically: Transparently reward high performers and address poor performers promptly to maintain motivation and performance standards.\n\n5. Avoid micromanagement: Give executives the autonomy to lead while providing clear expectations and monitoring results.\n\nRemember that these tips are based on the context provided and may not cover all aspects of effective CEO leadership.",
 'context': [(' management. Nuke as many of t

Navigate to the Weights and Biases "run" link to see how your LLM is performing!

```
View run at YOUR LINK HERE
https://wandb.ai/ai-bootcamp/Visibility%20Example%20-%20AIE3/runs/wnyjh58a?nw=nwuserkbrdek

#### ❓ Question #5:

What is the `model_name` from the WandB `root_span` trace?

gpt-3.5-turbo-0125

## Task 7: RAG Evaluation Using GPT-4



In [35]:
query = "What are some tips for being an effective CEO?"

response = retrieval_augmented_qa_pipeline.run_pipeline(query)

print(response["response"])

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response and the provided context
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Context: {context}
Response: {response}"""

try:
    chat_openai = ChatOpenAI(model_name="gpt-4-turbo")
except:
    chat_openai = ChatOpenAI()

evaluator_system_prompt = SystemRolePrompt(evaluator_system_template)
evaluation_prompt = UserRolePrompt(evaluation_template)

messages = [
    evaluator_system_prompt.create_message(format=False),
    evaluation_prompt.create_message(
        input=query,
        context="\n".join([context[0] for context in response["context"]]),
        response=response["response"]
    ),
]

chat_openai.run(messages, response_format={"type" : "json_object"})

Based on the context provided, here are some tips for being an effective CEO:

1. Focus on promoting and developing strong, sharp, aggressive, and ambitious director- or VP-level managers who will drive the company's success.
2. Identify and promote top 20 or 30 up-and-coming managers who show potential and give them clear responsibilities and missions.
3. Keep a direct line of communication with these top managers and have at most one executive between you and them to ensure effective leadership.
4. Clearly define the single most important thing your company needs to win at and put your best person in charge of achieving that goal.
5. Use incentives, such as stock-option grants with transparent performance metrics, to motivate and reward high performers within the organization.
6. Avoid micromanaging and empower your executives to make decisions and run their teams effectively.
7. Continuously gather data by asking questions, violating the chain of command if necessary, to stay inform

'{"clarity": "6", "faithfulness": "8", "correctness": "8"}'

# Conclusion

In this notebook, we've gone through the steps required to create your own simple RAQA application!

Please feel free to extend this as much as you'd like.

In [36]:
wandb.finish()